# ACG Seminar Assignment -- Neural Volume Compression  

Student: Anže Kristan  
class: Advanced Computer Graphics  
semester: Spring 2023/24  
school: Faculty of Computer and Information Science, University of Ljubljana  

#### imports and setup:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torchsummary import summary
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.ndimage import gaussian_filter
from os import path as ospath

In [ ]:
folder_path = "todo"
files = {
    "tooth": {
        "file_name": "tooth_103x94x161_1x1x1_uint8.raw",
        "shape": [103, 94, 161],
        "dtype": np.uint8
    }
}

# from https://www.geeksforgeeks.org/reading-binary-files-in-python/

# Open the file in binary mode
def readRawFile(file_path, datatype, shape, doReshape=True, divideBy=2**8, gaussianFilter=False, cutoffLow=(0,0), cutoffHigh=(255,255)):
  array = np.fromfile(file_path, dtype=datatype).astype(np.float32) # read from file based on datatype
  array[array<cutoffLow[0]] = cutoffLow[1] # set values below cutoff[0] to the value of cutoff[1]
  array[array>cutoffHigh[0]] = cutoffHigh[1]
  array = np.divide(array, divideBy) # divide input by specified (to get input into range 0-1)
  if (doReshape):
    array = np.reshape(array, list(reversed(shape))).transpose() # need to make sure reshape takes the same order as the file all x->all y->all z
    if (gaussianFilter):
      array = gaussian_filter(array, sigma=0.1, radius=3)
  return array

#### Dataset class:

Dataset class to help with machine learning; sample it like an array with an index (int) and returns a sample corresponding to that index

In [ ]:
class Dataset:
  def __init__(self,samples,stride=[4,4,4],kernel_size=[8,8,8], returnSurroundingIndices=True):
    self.stride = stride # [x,y,z]
    self.kernel_size = kernel_size # single value treated as length of the side of a cube
    self.returnSurroundingIndices = returnSurroundingIndices
    self.orig_shape = samples.shape
    self.samples = self.pad_to_divisible_and_kernel(samples)

    self.unfold_dims = np.array([ # num samples in each dimension
        int(np.ceil((self.orig_shape[0] - self.kernel_size[0])/self.stride[0])+1),
        int(np.ceil((self.orig_shape[1] - self.kernel_size[1])/self.stride[1])+1),
        int(np.ceil((self.orig_shape[2] - self.kernel_size[2])/self.stride[2])+1)
    ])
    self.length =  np.prod(self.unfold_dims) # total num samples

  def pad_to_divisible_and_kernel(self, arr):
    # pad the array to be divisible by kernel size
    to_pad = np.remainder(arr.shape, self.kernel_size)
    to_pad = [self.kernel_size[x] - to_pad[x] if to_pad[x] > 0 else 0 for x in range(3)]
    arr_pd = np.pad(arr, ( (0, to_pad[0]), (0, to_pad[1]), (0, to_pad[2]) ), mode='constant', constant_values='0')

    if self.returnSurroundingIndices:
      # pad an extra kernel_size around array
      extra_pad = [[self.kernel_size[x],self.kernel_size[x]] for x in range(3)]
      arr_pd2 = torch.tensor(np.pad(arr_pd, extra_pad, mode='constant', constant_values=0))
    else:
      arr_pd2 = arr_pd
    return torch.Tensor(arr_pd2)


  def __getitem__(self,index):
    arr_coord = np.multiply(np.unravel_index(index, self.unfold_dims), self.stride)
    if self.returnSurroundingIndices:
      arr_coord = np.add(arr_coord, self.kernel_size) # skip initial kernel_size of padding
    ix = 0
    if self.returnSurroundingIndices:
      surrounding_indices = [-1, 0, 1]
      ret_arr = torch.empty((27, *self.kernel_size))
    else:
      surrounding_indices = [0]
      ret_arr = torch.empty((1, *self.kernel_size))
    for z in surrounding_indices:
      for y in surrounding_indices:
        for x in surrounding_indices:
          tmp_coords = np.add(arr_coord, np.multiply([x, y, z], self.kernel_size))
          # print(ix, tmp_coords, z, y, x)
          ret_arr[ix] = self.get_sub_matrix_from_coords(tmp_coords)
          ix += 1
    return ret_arr

  def get_sub_matrix_from_coords(self, coords):
    batch_start = coords
    # print(batch_start)
    batch_end = np.add(coords, self.kernel_size)
    # print(batch_end)
    ret_arr = self.samples[batch_start[0]:batch_end[0], batch_start[1]:batch_end[1], batch_start[2]:batch_end[2]]
    # print(ret_arr.shape)
    return ret_arr

  def __len__(self):
    return self.length

  def shuffle(self):
    pass

#### Fit function:

Define the fit function:

In [ ]:
from torch.optim import SGD, Adam
from torch.nn import MSELoss, CrossEntropyLoss, L1Loss
import datetime

def fit(model, dset_train, num_epochs=1, train_batches=1, optimFunc=Adam, lossFunc=MSELoss, folder_path=folder_path, verbose=2):
  # with help from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

  curr_best_loss = np.Infinity
  training_losses = []
  save_path = ospath.join(folder_path, model.name)

  optimizer = optimFunc(model.parameters(), lr=0.1)
  loss = lossFunc()
  model.train(True)
  start_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"starting training for model '{model.name}' at: {datetime.datetime.now()}")
  for i in range(num_epochs):
    t_losses = []

    for j in range(train_batches):
      x_train = dset_train[j]
      p = model(x_train)
      l = loss(p, x_train)

      optimizer.zero_grad()
      l.backward()
      optimizer.step()

      t_losses.append(l.item())

    curr_loss = np.mean(t_losses)
    training_losses.append(curr_loss)

    if curr_loss < curr_best_loss:
      curr_best_loss = curr_loss
      torch.save(model.state_dict(), save_path)
    if (verbose > 1):
      print(f"epoch {i+1}/{num_epochs} at time {datetime.datetime.now().time()}; current loss: {curr_loss}")
    elif (verbose == 1):
      print(f"epoch {i+1}/{num_epochs: <{10}}", end="\r")

  end_time = datetime.datetime.now()
  if (verbose > 0):
    print(f"end of training {num_epochs} epochs: {datetime.datetime.now()}; elapsed time: {end_time - start_time}")
  best_model = model
  best_model.load_state_dict(torch.load(save_path))

  return best_model, training_losses

#### Linear models:

##### AELinPass  
Try a pass linear model to make sure it works

In [ ]:
class autoEncLinPass(nn.Module):

  def __init__(self, name="linae1", kernel_size=[8,8,8]):
    super(autoEncLinPass, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      )
    self.decoder = nn.Sequential(
      nn.Unflatten(1, [1, *kernel_size]),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [ ]:
def trainAndOutputAutoEncPassModel(folder_path=folder_path, model=autoEncLinPass, kernel_size=[4,4,4], stride_size=[4,4,4], epochs=4, name="model1lin", vol_name="tooth", verbose=True):
  # to train model
  tooth_path=ospath.join(folder_path, files[vol_name]["file_name"])
  tooth_array=readRawFile(tooth_path, files[vol_name]["dtype"], files[vol_name]["shape"], cutoffLow=(105,0))#, gaussianFilter=True)
  model1toothDset = Dataset(tooth_array, stride=stride_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1 = model(name=name, kernel_size=kernel_size)

  # to output encoded representation
  if (verbose):
    print(f"starting encoding at: {datetime.datetime.now()}")
  toothEncDset = Dataset(tooth_array, stride=kernel_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1_encoded_reps = np.array([model1.encoder(toothEncDset[x]).detach().numpy().flatten() for x in range(toothEncDset.length)])
  model1_encoded_reps.reshape(-1).tofile(ospath.join(folder_path, f"{model1.name}.enc"), format="%<f")

  # output reconstructed model
  if (verbose):
      print(f"starting decoding at: {datetime.datetime.now()}")
  model1_reconstructed = np.empty(toothEncDset.samples.shape)
  for ix, enc in enumerate(model1_encoded_reps):
    tmp_dec = model1.decoder(torch.tensor(enc).reshape((1,-1))).detach().numpy()
    ixst = np.multiply(np.unravel_index(ix, toothEncDset.unfold_dims), toothEncDset.stride)
    ixed = np.add(ixst,toothEncDset.kernel_size)
    # print(ix, ixst, ixed)
    model1_reconstructed[ixst[0]:ixed[0], ixst[1]:ixed[1], ixst[2]:ixed[2]] = tmp_dec

  model1_reconstructed = model1_reconstructed[0:toothEncDset.orig_shape[0], 0:toothEncDset.orig_shape[1], 0:toothEncDset.orig_shape[2]]
  # model1_reconstructed = gaussian_filter(model1_reconstructed, sigma=1, radius=np.divide(kernel_size,2).astype(np.int32)) # filter output to get rid of some of the blockiness
  mse = np.mean(np.power((model1_reconstructed*255 - tooth_array*255),2)) # from https://stackoverflow.com/a/18047482
  print(f"MSE reconstructed - input: {mse}")
  model1_reconstructed = (model1_reconstructed*255).astype(np.uint8).transpose().reshape(-1)
  model1_reconstructed.tofile(ospath.join(folder_path, f"{model1.name}.raw"), format="%<hhf")

  return

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_name = "linae1pass_8_8"
model1 = autoEncLinPass

model1_epochs = 0 # 0 epochs just runs the encoder/decoder without training

# trainAndOutputAutoEncPassModel(folder_path=folder_path, model=model1, kernel_size=model1_kernel, stride_size=model1_stride, epochs=model1_epochs, name=model1_name, vol_name="tooth", verbose=True)

##### AELin1

In [ ]:
class autoEncLin1(nn.Module):

  def __init__(self, name="linae1", kernel_size=[8,8,8], enc_size=9):
    super(autoEncLin1, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=sample_size, out_features=int(sample_size/2)),
      nn.PReLU(), #use parametric relu for activation functions
      nn.Linear(int(sample_size/2), int(sample_size/4)),
      nn.PReLU(),
      nn.Linear(int(sample_size/4), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), enc_size),
      nn.PReLU(),
      )
    self.decoder = nn.Sequential(
      nn.Linear(enc_size, int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/4)),
      nn.PReLU(),
      nn.Linear(int(sample_size/4), int(sample_size/2)),
      nn.PReLU(),
      nn.Linear(int(sample_size/2), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [ ]:
def trainAndOutputAutoEnc1Model(folder_path=folder_path, model=autoEncLin1, kernel_size=[4,4,4], stride_size=[4,4,4], enc_size=4, epochs=4, name="model1lin", optim=SGD, loss=MSELoss, vol_name="tooth", verbose=0, load_model_weights=False):
  # to train model
  tooth_path=ospath.join(folder_path, files[vol_name]["file_name"])
  tooth_array=readRawFile(tooth_path, files[vol_name]["dtype"], files[vol_name]["shape"], cutoffLow=(100,0))#, gaussianFilter=True)
  model1toothDset = Dataset(tooth_array, stride=stride_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1 = model(name=name, kernel_size=kernel_size, enc_size=enc_size)
  if (load_model_weights):
    save_path = ospath.join(folder_path, model1.name)
    model1.load_state_dict(torch.load(save_path))
  model1_best, model1_losses = fit(model1, model1toothDset, num_epochs=epochs, train_batches=model1toothDset.length-1, optimFunc=optim, lossFunc=loss, folder_path=folder_path, verbose=verbose)

  # to output encoded representation
  if (verbose > 0):
    print(f"starting encoding at: {datetime.datetime.now()}")
  toothEncDset = Dataset(tooth_array, stride=kernel_size, kernel_size=kernel_size, returnSurroundingIndices=False)
  model1_encoded_reps = np.array([model1_best.encoder(toothEncDset[x]).detach().numpy().flatten().astype(np.float16) for x in range(toothEncDset.length)])
  model1_encoded_reps.reshape(-1).tofile(ospath.join(folder_path, f"{model1.name}.enc"), format="%<hf")

  # output reconstructed model
  if (verbose>0):
      print(f"starting decoding at: {datetime.datetime.now()}")
  model1_reconstructed = np.empty(toothEncDset.samples.shape)
  for ix, enc in enumerate(model1_encoded_reps):
    tmp_dec = model1_best.decoder(torch.tensor(enc.astype(np.float32)).reshape((1,-1))).detach().numpy()
    ixst = np.multiply(np.unravel_index(ix, toothEncDset.unfold_dims), toothEncDset.stride)
    ixed = np.add(ixst,toothEncDset.kernel_size)
    # print(ix, ixst, ixed)
    model1_reconstructed[ixst[0]:ixed[0], ixst[1]:ixed[1], ixst[2]:ixed[2]] = tmp_dec
  if (verbose>0):
      print(f"finish decoding at: {datetime.datetime.now()}")

  model1_reconstructed = model1_reconstructed[0:toothEncDset.orig_shape[0], 0:toothEncDset.orig_shape[1], 0:toothEncDset.orig_shape[2]]
  # model1_reconstructed = gaussian_filter(model1_reconstructed, sigma=1, radius=np.divide(kernel_size,2).astype(np.int32)) # filter output to get rid of some of the blockiness
  mse = np.mean((np.multiply(model1_reconstructed,255) - np.multiply(tooth_array,255))**2) # from https://stackoverflow.com/a/18047482
  print(f"MSE: {mse}")
  model1_reconstructed = (model1_reconstructed*255).astype(np.uint8).transpose().reshape(-1)
  model1_reconstructed.tofile(ospath.join(folder_path, f"{model1.name}.raw"), format="%<hhf")

  model1_decoder = model1_best
  model1_decoder.encoder = None
  torch.save(model1_decoder.state_dict(), ospath.join(folder_path, f"{model1.name}.dec"))

  return model1_losses

In [ ]:
ae_lin1_load_weights = True
ae_lin1_epochs = 0
times_ran = 4 # x 50 epoch

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 1
model1_name = f"linae1_8_8_{int(model1_enc_size)}"
model1 = autoEncLin1

model1_epochs = ae_lin1_epochs # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = ae_lin1_load_weights # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae1_8_8_1' at: 2024-06-07 11:29:28.326853
epoch 1/49 at time 11:29:38.042561; current loss: 0.012241605034499095
epoch 2/49 at time 11:29:50.937903; current loss: 0.012192569659117568
epoch 3/49 at time 11:29:58.089216; current loss: 0.012141133235392064
epoch 4/49 at time 11:30:06.619488; current loss: 0.012114875999025897
epoch 5/49 at time 11:30:13.717756; current loss: 0.012074670573570248
epoch 6/49 at time 11:30:21.922910; current loss: 0.012040309124047308
epoch 7/49 at time 11:30:31.452817; current loss: 0.011994036796476955
epoch 8/49 at time 11:30:38.604890; current loss: 0.01196519946526657
epoch 9/49 at time 11:30:46.961539; current loss: 0.011953594385962413
epoch 10/49 at time 11:30:55.316889; current loss: 0.011914332545025719
epoch 11/49 at time 11:31:03.870879; current loss: 0.0119211454421794
epoch 12/49 at time 11:31:11.087163; current loss: 0.011877693621400788
epoch 13/49 at time 11:31:19.303560; current loss: 0.011853784167982015
ep

count    49.000000
mean      0.011906
std       0.000178
min       0.011559
25%       0.011802
50%       0.011878
75%       0.012048
max       0.012242
dtype: float64

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 3
model1_name = f"linae1_8_8_{int(model1_enc_size)}"
model1 = autoEncLin1

model1_epochs = ae_lin1_epochs # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = ae_lin1_load_weights # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae1_8_8_3' at: 2024-06-07 11:36:04.913107
epoch 1/49 at time 11:36:12.524960; current loss: 0.003937267553219117
epoch 2/49 at time 11:36:20.630453; current loss: 0.003915317802647959
epoch 3/49 at time 11:36:28.964589; current loss: 0.003889330925812776
epoch 4/49 at time 11:36:36.188570; current loss: 0.0038662638063148478
epoch 5/49 at time 11:36:44.657457; current loss: 0.0038445766860840686
epoch 6/49 at time 11:36:51.627873; current loss: 0.00382180291097437
epoch 7/49 at time 11:36:59.860814; current loss: 0.00380405185302841
epoch 8/49 at time 11:37:06.856378; current loss: 0.003772916415880879
epoch 9/49 at time 11:37:15.504108; current loss: 0.003753191327936213
epoch 10/49 at time 11:37:22.869048; current loss: 0.0037358023342690346
epoch 11/49 at time 11:37:30.927041; current loss: 0.0037157232889023038
epoch 12/49 at time 11:37:38.759476; current loss: 0.0036946721630491826
epoch 13/49 at time 11:37:46.592717; current loss: 0.003671813439345

count    49.000000
mean      0.003355
std       0.000366
min       0.002767
25%       0.003014
50%       0.003379
75%       0.003672
max       0.003937
dtype: float64

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 5
model1_name = f"linae1_8_8_{int(model1_enc_size)}"
model1 = autoEncLin1

model1_epochs = ae_lin1_epochs # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = ae_lin1_load_weights # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae1_8_8_5' at: 2024-06-07 11:42:32.043044
epoch 1/49 at time 11:42:40.726020; current loss: 0.0016859530235841223
epoch 2/49 at time 11:42:47.720541; current loss: 0.0016810546274816525
epoch 3/49 at time 11:42:58.111158; current loss: 0.001676681993241361
epoch 4/49 at time 11:43:05.173913; current loss: 0.0016661041352288865
epoch 5/49 at time 11:43:13.683895; current loss: 0.0016603869294785024
epoch 6/49 at time 11:43:20.890925; current loss: 0.0016564296459121073
epoch 7/49 at time 11:43:29.444276; current loss: 0.0016437510402228786
epoch 8/49 at time 11:43:36.694489; current loss: 0.0016388240233525023
epoch 9/49 at time 11:43:45.154498; current loss: 0.0016343912878222673
epoch 10/49 at time 11:43:52.844232; current loss: 0.0016279164677596952
epoch 11/49 at time 11:44:00.597232; current loss: 0.0016213531655147777
epoch 12/49 at time 11:44:08.729932; current loss: 0.001614038339990806
epoch 13/49 at time 11:44:15.906550; current loss: 0.00161138

count    49.000000
mean      0.001571
std       0.000056
min       0.001486
25%       0.001530
50%       0.001560
75%       0.001611
max       0.001686
dtype: float64

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 7
model1_name = f"linae1_8_8_{int(model1_enc_size)}"
model1 = autoEncLin1

model1_epochs = ae_lin1_epochs # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = ae_lin1_load_weights # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae1_8_8_7' at: 2024-06-07 11:49:02.235095
epoch 1/49 at time 11:49:10.675616; current loss: 0.0018950033703691684
epoch 2/49 at time 11:49:17.771508; current loss: 0.001887446522323305
epoch 3/49 at time 11:49:26.333236; current loss: 0.0018792999016516085
epoch 4/49 at time 11:49:33.740400; current loss: 0.0018719941295304119
epoch 5/49 at time 11:49:42.061426; current loss: 0.0018638042131196024
epoch 6/49 at time 11:49:49.700157; current loss: 0.0018556899880766873
epoch 7/49 at time 11:49:57.470675; current loss: 0.0018487900015389913
epoch 8/49 at time 11:50:05.889737; current loss: 0.001841413877159157
epoch 9/49 at time 11:50:13.286567; current loss: 0.0018338862416870678
epoch 10/49 at time 11:50:21.862156; current loss: 0.0018267869864330142
epoch 11/49 at time 11:50:29.047366; current loss: 0.0018197948661752834
epoch 12/49 at time 11:50:37.534497; current loss: 0.0018129960358311034
epoch 13/49 at time 11:50:44.549640; current loss: 0.00180545

count    49.000000
mean      0.001741
std       0.000082
min       0.001620
25%       0.001671
50%       0.001731
75%       0.001805
max       0.001895
dtype: float64

##### AELin2

In [ ]:
class autoEncLin2(nn.Module):

  def __init__(self, name="linae2", kernel_size=[8,8,8], enc_size=9):
    super(autoEncLin2, self).__init__()
    sample_size = np.prod(kernel_size)
    self.encoder = nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=sample_size, out_features=int(sample_size/8)),
      nn.PReLU(), #use parametric relu for activation functions
      nn.Linear(int(sample_size/8), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/12)),
      nn.PReLU(),
      nn.Linear(int(sample_size/12), enc_size),
      nn.PReLU(),
      )
    self.decoder = nn.Sequential(
      nn.Linear(enc_size, int(sample_size/12)),
      nn.PReLU(),
      nn.Linear(int(sample_size/12), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size/8)),
      nn.PReLU(),
      nn.Linear(int(sample_size/8), int(sample_size)),
      nn.Unflatten(1, kernel_size),
      nn.Sigmoid(),
      )
    self.name = name

  def get_encoding(self, x):
    return self.encoder(x)

  def get_decoding(self, c):
    return self.decoder(c)

  def forward(self, x):
    c = self.encoder(x)
    y = self.decoder(c)
    return y

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 5
model1_name = f"linae2_8_8_{model1_enc_size}"
model1 = autoEncLin2

model1_epochs = 0 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = True # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae2_8_8_5' at: 2024-06-07 12:42:25.716701
epoch 1/50 at time 12:42:32.508813; current loss: 0.008250118583926326
epoch 2/50 at time 12:42:38.802978; current loss: 0.00822546267544442
epoch 3/50 at time 12:42:44.612602; current loss: 0.008197530948051309
epoch 4/50 at time 12:42:52.423414; current loss: 0.008162431728912196
epoch 5/50 at time 12:42:58.813913; current loss: 0.00812143541303491
epoch 6/50 at time 12:43:05.810489; current loss: 0.008071156316751819
epoch 7/50 at time 12:43:11.565482; current loss: 0.008031430634942384
epoch 8/50 at time 12:43:18.697559; current loss: 0.007979225988696418
epoch 9/50 at time 12:43:24.439635; current loss: 0.007928454264928065
epoch 10/50 at time 12:43:31.843367; current loss: 0.00787019931321347
epoch 11/50 at time 12:43:37.599003; current loss: 0.007797171473719916
epoch 12/50 at time 12:43:44.065187; current loss: 0.007725864063646955
epoch 13/50 at time 12:43:50.477234; current loss: 0.007662549344243725
ep

count    50.000000
mean      0.006290
std       0.001437
min       0.004566
25%       0.004772
50%       0.006739
75%       0.007644
max       0.008250
dtype: float64

In [ ]:
model1_kernel = [8]*3
model1_stride = [8]*3
model1_enc_size = 7
model1_name = f"linae2_8_8_{model1_enc_size}"
model1 = autoEncLin2

model1_epochs = 0 # 0 epochs just runs the encoder/decoder without training
model1_optim = SGD  #Adam#SGD
model1_loss = MSELoss   #CrossEntropyLoss#MSELoss
model1_load_weights = True # if changing anything above this line (especially kernel,stride,enc_size,name), set load_model_weights to False

model1_losses = trainAndOutputAutoEnc1Model(folder_path, model1, model1_kernel, model1_stride, model1_enc_size, model1_epochs, model1_name, model1_optim, model1_loss, vol_name="tooth", verbose=2, load_model_weights=model1_load_weights)
pd.Series(model1_losses).describe()

starting training for model 'linae2_8_8_7' at: 2024-06-07 12:56:32.933052
epoch 1/50 at time 12:56:39.018363; current loss: 0.002948944412057925
epoch 2/50 at time 12:56:46.119195; current loss: 0.002904636784904919
epoch 3/50 at time 12:56:51.944218; current loss: 0.002863972377390452
epoch 4/50 at time 12:56:59.255829; current loss: 0.002833395896619855
epoch 5/50 at time 12:57:05.069530; current loss: 0.0028041937976609668
epoch 6/50 at time 12:57:12.365060; current loss: 0.0027694544856208163
epoch 7/50 at time 12:57:18.038568; current loss: 0.002741970080301734
epoch 8/50 at time 12:57:25.043831; current loss: 0.0027180876553455804
epoch 9/50 at time 12:57:30.960680; current loss: 0.0026962456078991072
epoch 10/50 at time 12:57:37.716902; current loss: 0.002676496143568044
epoch 11/50 at time 12:57:44.278092; current loss: 0.0026570286640081294
epoch 12/50 at time 12:57:50.450236; current loss: 0.002636798548734244
epoch 13/50 at time 12:57:57.449242; current loss: 0.0026225277062

count    50.000000
mean      0.002518
std       0.000170
min       0.002310
25%       0.002385
50%       0.002471
75%       0.002619
max       0.002949
dtype: float64